In [34]:
import pandas as pd
import os
import numpy as np

In [35]:
print(os.getcwd())

/home/leon/Documents/Studienarbeit/EvalFramework/notebooks


In [36]:
train_df = pd.read_csv("../data/datasets/gtsrb/Train.csv")
print(train_df)

       Width  Height  Roi.X1  Roi.Y1  Roi.X2  Roi.Y2  ClassId  \
0         27      26       5       5      22      20       20   
1         28      27       5       6      23      22       20   
2         29      26       6       5      24      21       20   
3         28      27       5       6      23      22       20   
4         28      26       5       5      23      21       20   
...      ...     ...     ...     ...     ...     ...      ...   
39204     52      56       5       6      47      51       42   
39205     56      58       5       5      51      53       42   
39206     58      62       5       6      53      57       42   
39207     63      69       5       7      58      63       42   
39208     68      69       7       6      62      63       42   

                                 Path  
0      Train/20/00020_00000_00000.png  
1      Train/20/00020_00000_00001.png  
2      Train/20/00020_00000_00002.png  
3      Train/20/00020_00000_00003.png  
4      Train/20/000

In [37]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39209 entries, 0 to 39208
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Width    39209 non-null  int64 
 1   Height   39209 non-null  int64 
 2   Roi.X1   39209 non-null  int64 
 3   Roi.Y1   39209 non-null  int64 
 4   Roi.X2   39209 non-null  int64 
 5   Roi.Y2   39209 non-null  int64 
 6   ClassId  39209 non-null  int64 
 7   Path     39209 non-null  object
dtypes: int64(7), object(1)
memory usage: 2.4+ MB


# The YoloFileFormat:
YOLO format, with one *.txt file per image (if no objects in image, no *.txt file is required). The *.txt file specifications are:
- One row per object
- Each row is class x_center y_center width height format.
- Box coordinates must be in normalized xywh format (from 0 - 1). If your boxes are in pixels, divide x_center and width by image width, and y_center and height by image height.
- Class numbers are zero-indexed (start from 0).
![](https://user-images.githubusercontent.com/26833433/91506361-c7965000-e886-11ea-8291-c72b98c25eec.jpg)
The label file corresponding to the above image contains 2 persons (class 0) and a tie (class 27):
![](https://user-images.githubusercontent.com/26833433/112467037-d2568c00-8d66-11eb-8796-55402ac0d62f.png)

In [38]:
# Calculate Center of ROI
test_row = train_df.loc[0]
print(test_row)

Width                                  27
Height                                 26
Roi.X1                                  5
Roi.Y1                                  5
Roi.X2                                 22
Roi.Y2                                 20
ClassId                                20
Path       Train/20/00020_00000_00000.png
Name: 0, dtype: object


In [39]:
# The center of rectangle is the midpoint of the diagonal end points of rectangle
x_center = (test_row.loc["Roi.X1"] + test_row.loc["Roi.X2"]) // 2
y_center = (test_row.loc["Roi.Y1"] + test_row.loc["Roi.Y2"]) // 2
box_width = test_row.loc["Roi.X1"] + test_row.loc["Roi.X2"]
box_height = test_row.loc["Roi.Y1"] + test_row.loc["Roi.Y2"]
print(x_center, y_center, box_height, box_width)

13 12 25 27


In [40]:
# Normalize Coordinates (0-1)
train_df["Roi.X1"] /= train_df["Width"]
train_df["Roi.X2"] /= train_df["Width"]
train_df["Roi.Y1"] /= train_df["Height"]
train_df["Roi.Y2"] /= train_df["Height"]
print(train_df)

       Width  Height    Roi.X1    Roi.Y1    Roi.X2    Roi.Y2  ClassId  \
0         27      26  0.185185  0.192308  0.814815  0.769231       20   
1         28      27  0.178571  0.222222  0.821429  0.814815       20   
2         29      26  0.206897  0.192308  0.827586  0.807692       20   
3         28      27  0.178571  0.222222  0.821429  0.814815       20   
4         28      26  0.178571  0.192308  0.821429  0.807692       20   
...      ...     ...       ...       ...       ...       ...      ...   
39204     52      56  0.096154  0.107143  0.903846  0.910714       42   
39205     56      58  0.089286  0.086207  0.910714  0.913793       42   
39206     58      62  0.086207  0.096774  0.913793  0.919355       42   
39207     63      69  0.079365  0.101449  0.920635  0.913043       42   
39208     68      69  0.102941  0.086957  0.911765  0.913043       42   

                                 Path  
0      Train/20/00020_00000_00000.png  
1      Train/20/00020_00000_00001.png  
2  

In [41]:
print(len(train_df))
# train_converted = np.empty((len(train_df), 5))
for index, row in train_df.iterrows():
    print(row)
    break
train_converted = np.array([
    [
        row.loc["ClassId"],
        (row.loc["Roi.X1"] + row.loc["Roi.X2"]) / 2.0,
        (row.loc["Roi.Y1"] + row.loc["Roi.Y2"]) / 2.0,
        row.loc["Roi.X1"] + row.loc["Roi.X2"],
        row.loc["Roi.Y1"] + row.loc["Roi.Y2"]
    ] for index, row in train_df.iterrows()])
print(train_converted.shape)
print(train_converted)

39209
Width                                  27
Height                                 26
Roi.X1                           0.185185
Roi.Y1                           0.192308
Roi.X2                           0.814815
Roi.Y2                           0.769231
ClassId                                20
Path       Train/20/00020_00000_00000.png
Name: 0, dtype: object
(39209, 5)
[[20.          0.5         0.48076923  1.          0.96153846]
 [20.          0.5         0.51851852  1.          1.03703704]
 [20.          0.51724138  0.5         1.03448276  1.        ]
 ...
 [42.          0.5         0.50806452  1.          1.01612903]
 [42.          0.5         0.50724638  1.          1.01449275]
 [42.          0.50735294  0.5         1.01470588  1.        ]]


In [47]:
for index, row in train_df.iterrows():
    if row.loc["ClassId"] == train_converted[index][0] and (row.loc["Roi.X1"] + row.loc["Roi.X2"]) / 2 == train_converted[index][1] and (row.loc["Roi.Y1"] + row.loc["Roi.Y2"]) / 2 == train_converted[index][2] and row.loc["Roi.X1"] + row.loc["Roi.X2"] == train_converted[index][3] and row.loc["Roi.Y1"] + row.loc["Roi.Y2"] == train_converted[index][4]:
        continue
    else:
        print("row false values")
    break

In [42]:
from shutil import move, copy2
from os import mkdir, listdir
gtsrb_root = "../data/datasets/gtsrb/"

mkdir(f"{gtsrb_root}train/images/")
#move(f"{gtsrb_root}train/0/00000_00000_00000.png", f"{gtsrb_root}train/images/")

for folder in listdir(f"{gtsrb_root}train/"):
    if folder == "images":
        continue
    for image in listdir(f"{gtsrb_root}train/{folder}/"):
        move(f"{gtsrb_root}train/{folder}/{image}", f"{gtsrb_root}train/images/")
    os.rmdir(f"{gtsrb_root}train/{folder}")

In [44]:
print(len(listdir(f"{gtsrb_root}train/images/")))
print(len(listdir(f"{gtsrb_root}test/")))

39209
12631


In [48]:
dfs = [pd.read_csv(f"{gtsrb_root}Train.csv"), pd.read_csv(f"{gtsrb_root}Test.csv")]
print(dfs[0])

       Width  Height  Roi.X1  Roi.Y1  Roi.X2  Roi.Y2  ClassId  \
0         27      26       5       5      22      20       20   
1         28      27       5       6      23      22       20   
2         29      26       6       5      24      21       20   
3         28      27       5       6      23      22       20   
4         28      26       5       5      23      21       20   
...      ...     ...     ...     ...     ...     ...      ...   
39204     52      56       5       6      47      51       42   
39205     56      58       5       5      51      53       42   
39206     58      62       5       6      53      57       42   
39207     63      69       5       7      58      63       42   
39208     68      69       7       6      62      63       42   

                                 Path  
0      Train/20/00020_00000_00000.png  
1      Train/20/00020_00000_00001.png  
2      Train/20/00020_00000_00002.png  
3      Train/20/00020_00000_00003.png  
4      Train/20/000

In [81]:
# mkdir(f"{gtsrb_root}train/labels/")
print(f"{gtsrb_root}train/labels/{dfs[0].loc[0].loc['Path'].split('/')[2][:-4]}.txt")
f = open(f"{gtsrb_root}train/labels/{dfs[0].loc[0].loc['Path'].split('/')[2][:-4]}.txt", "w+")
# f.write(f"{int(train_converted[0][0])} {train_converted[0][1]} {train_converted[0][2]} {train_converted[0][3]} {train_converted[0][4]}")
f.write(f"{int(train_converted[0][0])} {' '.join(map(str, train_converted[0][1:]))}")
print([x for x in map(str, train_converted[0])])
f.close()

../data/datasets/gtsrb/train/labels/00020_00000_00000.txt
['20.0', '0.5', '0.4807692307692308', '1.0', '0.9615384615384616']


In [63]:
for index, row in train_df.iterrows():
    current_subset_id, _, current_image = row.loc["Path"].split("/")
    print(current_image, current_subset_id)
    break

00020_00000_00000.png Train
